In [71]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# Let's start by loading the datasets
import pandas as pd
# Load train data
try:
    train_df = pd.read_csv('train.csv')
except:
    train_df = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/test/train.csv')
# Load store data
try:
    store_df = pd.read_csv('store.csv')
except:
    store_df = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/test/store.csv')
# Load test data
try:
    test_df = pd.read_csv('test.csv')
except:
    test_df = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/test/test.csv')

C:\Users\rlaeo\AppData\Local\Temp\ipykernel_9276\3257281094.py:14: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('C:/Users/rlaeo/OneDrive/바탕 화면/test/train.csv')


In [72]:
# Merge the train and store dataframes
train_df = pd.merge(train_df, store_df, on='Store', how='left')

# Merge the test and store dataframes
test_df = pd.merge(test_df, store_df, on='Store', how='left')

# Display the first few rows of each merged dataframe
train_df.head(), test_df.head()

(   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday   
 0      1          5  2015-07-31   5263        555     1      1            0  \
 1      2          5  2015-07-31   6064        625     1      1            0   
 2      3          5  2015-07-31   8314        821     1      1            0   
 3      4          5  2015-07-31  13995       1498     1      1            0   
 4      5          5  2015-07-31   4822        559     1      1            0   
 
    SchoolHoliday StoreType Assortment  CompetitionDistance   
 0              1         c          a               1270.0  \
 1              1         a          a                570.0   
 2              1         a          a              14130.0   
 3              1         c          c                620.0   
 4              1         a          a              29910.0   
 
    CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2   
 0                        9.0                    2008.0       0  \
 1  

In [73]:
train_missing = train_df.isnull().sum()
test_missing = test_df.isnull().sum()

train_missing[train_missing > 0], test_missing[test_missing > 0]

(CompetitionDistance            2642
 CompetitionOpenSinceMonth    323348
 CompetitionOpenSinceYear     323348
 Promo2SinceWeek              508031
 Promo2SinceYear              508031
 PromoInterval                508031
 dtype: int64,
 Open                            11
 CompetitionDistance             96
 CompetitionOpenSinceMonth    15216
 CompetitionOpenSinceYear     15216
 Promo2SinceWeek              17232
 Promo2SinceYear              17232
 PromoInterval                17232
 dtype: int64)

In [74]:
# Fill missing values

# For 'CompetitionDistance', we fill missing values with the median distance
train_df['CompetitionDistance'].fillna(train_df['CompetitionDistance'].median(), inplace=True)
test_df['CompetitionDistance'].fillna(test_df['CompetitionDistance'].median(), inplace=True)

# For 'CompetitionOpenSinceMonth' and 'CompetitionOpenSinceYear', we fill missing values with the most frequent value
train_df['CompetitionOpenSinceMonth'].fillna(train_df['CompetitionOpenSinceMonth'].mode()[0], inplace=True)
train_df['CompetitionOpenSinceYear'].fillna(train_df['CompetitionOpenSinceYear'].mode()[0], inplace=True)

test_df['CompetitionOpenSinceMonth'].fillna(test_df['CompetitionOpenSinceMonth'].mode()[0], inplace=True)
test_df['CompetitionOpenSinceYear'].fillna(test_df['CompetitionOpenSinceYear'].mode()[0], inplace=True)

# For 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', we fill missing values with zero
promo_cols = ['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']
train_df[promo_cols] = train_df[promo_cols].fillna(0)
test_df[promo_cols] = test_df[promo_cols].fillna(0)

# For 'Open' column in test dataset, fill missing values with 1
test_df['Open'].fillna(1, inplace=True)


# Check if there are any missing values left
train_missing = train_df.isnull().sum()
test_missing = test_df.isnull().sum()

train_missing[train_missing > 0], test_missing[test_missing > 0]
test_df.drop('Date', axis=1, inplace=True)



In [75]:
# One-hot encode 'StateHoliday', 'StoreType', 'Assortment' columns
train_df = pd.get_dummies(train_df, columns=['StateHoliday', 'StoreType', 'Assortment'])
test_df = pd.get_dummies(test_df, columns=['StateHoliday', 'StoreType', 'Assortment'])

# Handle 'PromoInterval' column
# First, let's see what unique values it has
print("Unique values in 'PromoInterval' column:")
print(train_df['PromoInterval'].unique())

# It seems like 'PromoInterval' contains either 0 (for no promotion) or a string with comma-separated months.
# Let's create separate binary columns for each possible month in the 'PromoInterval'

# Months
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# For each month, create a new column in train and test dataframes
for month in months:
    train_df['PromoInterval_' + month] = train_df['PromoInterval'].apply(lambda x: 1 if month in str(x) else 0)
    test_df['PromoInterval_' + month] = test_df['PromoInterval'].apply(lambda x: 1 if month in str(x) else 0)

# Now that we've extracted the necessary information from the 'PromoInterval' column, we can drop it
train_df.drop('PromoInterval', axis=1, inplace=True)
test_df.drop('PromoInterval', axis=1, inplace=True)

# Check the updated dataframes
train_df.head(), test_df.head()


Unique values in 'PromoInterval' column:
[0 'Jan,Apr,Jul,Oct' 'Feb,May,Aug,Nov' 'Mar,Jun,Sept,Dec']


(   Store  DayOfWeek        Date  Sales  Customers  Open  Promo  SchoolHoliday   
 0      1          5  2015-07-31   5263        555     1      1              1  \
 1      2          5  2015-07-31   6064        625     1      1              1   
 2      3          5  2015-07-31   8314        821     1      1              1   
 3      4          5  2015-07-31  13995       1498     1      1              1   
 4      5          5  2015-07-31   4822        559     1      1              1   
 
    CompetitionDistance  CompetitionOpenSinceMonth  ...  PromoInterval_Mar   
 0               1270.0                        9.0  ...                  0  \
 1                570.0                       11.0  ...                  0   
 2              14130.0                       12.0  ...                  0   
 3                620.0                        9.0  ...                  0   
 4              29910.0                        4.0  ...                  0   
 
    PromoInterval_Apr  PromoInterval

In [76]:
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler
scaler = MinMaxScaler()

# List of numerical columns
num_cols = ['CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']

# Apply the scaler to the numerical columns
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols]) # Note: we use transform() here, not fit_transform()

# Check the updated dataframes
train_df.head(), test_df.head()


(   Store  DayOfWeek        Date  Sales  Customers  Open  Promo  SchoolHoliday   
 0      1          5  2015-07-31   5263        555     1      1              1  \
 1      2          5  2015-07-31   6064        625     1      1              1   
 2      3          5  2015-07-31   8314        821     1      1              1   
 3      4          5  2015-07-31  13995       1498     1      1              1   
 4      5          5  2015-07-31   4822        559     1      1              1   
 
    CompetitionDistance  CompetitionOpenSinceMonth  ...  PromoInterval_Mar   
 0             0.016482                   0.727273  ...                  0  \
 1             0.007252                   0.909091  ...                  0   
 2             0.186050                   1.000000  ...                  0   
 3             0.007911                   0.727273  ...                  0   
 4             0.394119                   0.272727  ...                  0   
 
    PromoInterval_Apr  PromoInterval

In [77]:
# Separate target variable 'Sales' from the rest of the training dataset
X_train = train_df.drop(['Sales', 'Customers'], axis=1) # We also drop 'Customers' as it's not present in the test dataset
y_train = train_df['Sales']

# Ensure that train and test datasets have the same columns
missing_cols = set(X_train.columns) - set(test_df.columns)
for c in missing_cols:
    test_df[c] = 0
test_df = test_df[X_train.columns]

# Check the shapes of the datasets
X_train.shape, y_train.shape, test_df.shape


((1017209, 36), (1017209,), (41088, 36))

In [78]:
#imports
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


In [81]:

import torch
import subprocess

def GPU():
    if torch.cuda.is_available() == True:
        device = 'cuda'
        templist = [1, 2, 3]
        templist = torch.FloatTensor(templist).to(device)
        print("Cuda torch working : ", end="")
        print(templist.is_cuda)
        print("current device no. : ", end="")
        print(torch.cuda.current_device())
        print("GPU device count : ", end="")
        print(torch.cuda.device_count())
        print("GPU name : ", end="")
        print(torch.cuda.get_device_name(0))
        print("device : ", device)
        # Execute the nvidia-smi command using subprocess
        try:
            output = subprocess.check_output(['nvidia-smi']).decode('utf-8')
            print("nvidia-smi output:")
            print(output)
        except (subprocess.CalledProcessError, FileNotFoundError) as e:
            print("Error executing nvidia-smi command:", str(e))
    elif torch.backends.mps.is_available() == True:
        print("Apple device detected\nActivating Apple Silicon GPU")
        device = torch.device("mps")
    else:
        print("cant use gpu , activating cpu")
        device = 'cpu'

    return device
device = GPU()
device = 'cpu'
print(device)

cant use gpu , activating cpu
cpu


In [84]:
# Convert the dataframes into PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device) # reshape to (n_samples, 1)
X_test_tensor = torch.tensor(test_df.values, dtype=torch.float32).to(device)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses

# Define the FFN model using TensorFlow
def build_ffn_model(input_dim):
    model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(input_dim,)),
        layers.Dropout(0.2),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.BatchNormalization(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.BatchNormalization(),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.2),
        layers.BatchNormalization(),
        layers.Dense(1)
    ])
    return model

# Create the model
model = build_ffn_model(X_train_tensor.shape[1])

# Compile the model
optimizer = optimizers.Adam()
loss_function = losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss_function)

# Train the model
history = model.fit(train_loader, validation_data=val_loader, epochs=100, verbose=1)

# Plot the training and validation losses
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Training and Validation Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_graph.png')
plt.show()
